In [1]:
from prompt_encoder import PromptEncoder, TwoWayTransformer
import torch

In [2]:
# one = torch.ones(3,3,3)
# y_embed = one.cumsum(dim=0)
# x_embed = one.cumsum(dim=1)
# z_embed = one.cumsum(dim=2)
# print(y_embed, x_embed,z_embed)

In [3]:
import numpy as np
seg = torch.ones(1,128,128,128)
l = len(torch.where(seg == 1)[0])
print(l)
sample = np.random.choice(np.arange(l), 10, replace=True) # 從範圍為 [0, l) 的整數中隨機選取 10 個數字（可能有重複）
print(sample)
x = torch.where(seg == 1)[1][sample].unsqueeze(1)
y = torch.where(seg == 1)[3][sample].unsqueeze(1)
z = torch.where(seg == 1)[2][sample].unsqueeze(1)
print(z)
point_coord = torch.cat([x, y, z], dim=1).unsqueeze(1).float() 

foo = torch.randn(1,20,3)
point_coord = point_coord.transpose(0,1)
point_coord = torch.cat([point_coord,foo],dim=1)
print(point_coord.size())



2097152
[ 935811  441419  912402 1867605   35191  184341  521385  389362  137602
 2095285]
tensor([[ 15],
        [120],
        [ 88],
        [126],
        [ 18],
        [ 32],
        [105],
        [ 97],
        [ 51],
        [113]])
torch.Size([1, 30, 3])


In [4]:
foo_feature = torch.randn(1,256,32,32,32) # 1, 256, ?, ?, ?
prompt_encoder = PromptEncoder(transformer=TwoWayTransformer(depth=2,
                                                                 embedding_dim=256,
                                                                 mlp_dim=2048,
                                                                 num_heads=8))
prompt_encoder.to("cpu")
patch_size=128
ans = prompt_encoder(foo_feature, point_coord, [patch_size, patch_size, patch_size]) # ?, [1,30,3], [128,128,128]

ans.size() # 1, 256, 32, 32, 32

coords torch.Size([32, 32, 32, 3])
pe torch.Size([32, 32, 32, 256])
image_pe torch.Size([1, 256, 32, 32, 32])
送進transformer的三個參數image_embeddings, image_pe, point_coord torch.Size([1, 256, 32, 32, 32]) torch.Size([1, 256, 32, 32, 32]) torch.Size([1, 1, 1, 30, 3])
===init===
image_embedding init torch.Size([1, 256, 32, 32, 32])
point_coord init torch.Size([1, 1, 1, 30, 3])

point_embedding after grid sample torch.Size([1, 256, 1, 1, 30])
point_pe after grid sample torch.Size([1, 256, 1, 1, 30])

        之所以維度由[1,256,32,32,32]變成[1,256,1,1,30], 是因為point_coord [1,1,1,30,3]中包含了30個xyz的座標(已正規化到-1~1之間)
        定位了在image_embedding中的30個位置(維度中為32的D*H*W), 並對原始在對應image_embedding空間上的特徵進行插值(僅限這30個點)
        因此結果會是[1,256,1,1,30], 最後一個維度代表其中某一個通道在這30個點中的特徵值
        

        接下來squeeze去除1維度
        
point_embedding after squeeze torch.Size([1, 256, 30])
point_pe after squeeze torch.Size([1, 256, 30])

        permute後, 現在我們有包含了點座標資訊的point_embedding特徵以及包含了點座標資訊的point_pe(一個固定的位置編碼矩陣)
        
point_embedd

torch.Size([1, 256, 32, 32, 32])

In [12]:
import torch
import math

def positional_encoding(d_model, height, width, depth):
    """
    :param d_model: the dimension of embeddings.
    :param height: height of the input.
    :param width: width of the input.
    :param depth: depth of the input.
    :return: position encoding tensor.
    """
    assert d_model % 3 == 0

    # Create a position encoding tensor
    pe = torch.zeros([1, d_model, height, width, depth])

    # Even indices
    div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
    
    pos_w = torch.arange(0., width).unsqueeze(1)
    pe[:, 0::3, :, :, :] = torch.sin(pos_w * div_term[:width]).unsqueeze(0).unsqueeze(2).unsqueeze(4)

    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[:, 1::3, :, :, :] = torch.sin(pos_h * div_term[:height]).unsqueeze(0).unsqueeze(3).unsqueeze(4)

    pos_d = torch.arange(0., depth).unsqueeze(1)
    pe[:, 2::3, :, :, :] = torch.sin(pos_d * div_term[:depth]).unsqueeze(0).unsqueeze(2).unsqueeze(3)

    return pe

# Create a position encoding tensor with size [32,32,32]
pe = positional_encoding(96, 32, 32, 32)
print(pe.size())
pe

torch.Size([1, 96, 32, 32, 32])


tensor([[[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.